#Actividad 1-2-3-4.

# Se acordó que esta actividad reemplaza las correspondientes al módulo 8.
* Debido a falta de información o por estar incompletas.
* El dataset fue proporcionado por el docente.

In [535]:
!pip install -U pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import when
from pyspark.sql import functions as F
from pyspark.sql.functions import countDistinct

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [536]:
spark = SparkSession.builder.appName(
    "HeartDiseaseClassification"
).getOrCreate()

In [537]:
# Read the data
df = spark.read.option(
    "delimiter", " "
).csv('heart.dat', inferSchema=True, header=False)

In [538]:
df.show(3, truncate=False)

+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
|_c0 |_c1|_c2|_c3  |_c4  |_c5|_c6|_c7  |_c8|_c9|_c10|_c11|_c12|_c13|
+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
|70.0|1.0|4.0|130.0|322.0|0.0|2.0|109.0|0.0|2.4|2.0 |3.0 |3.0 |2   |
|67.0|0.0|3.0|115.0|564.0|0.0|2.0|160.0|0.0|1.6|2.0 |0.0 |7.0 |1   |
|57.0|1.0|2.0|124.0|261.0|0.0|0.0|141.0|0.0|0.3|1.0 |0.0 |7.0 |2   |
+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
only showing top 3 rows



In [539]:
# Rename the columns
new_column_names = [
    'año',
    'sexo',
    'tipoDolor',
    'presionDescanso', 
    'colesterol',
    'azucarSangreL120', 
    'electrocardioDescanso',
    'frecCardMax', 
    'angina',
    'depresionST',
    'pendienteST', 
    'numVasos',
    'thal'
]

for i in range(len(new_column_names)):
    df = df.withColumnRenamed(df.columns[i], new_column_names[i])

df = df.drop(df.columns[-1])
df.show(5, truncate=False)

+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+
|año |sexo|tipoDolor|presionDescanso|colesterol|azucarSangreL120|electrocardioDescanso|frecCardMax|angina|depresionST|pendienteST|numVasos|thal|
+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+
|70.0|1.0 |4.0      |130.0          |322.0     |0.0             |2.0                  |109.0      |0.0   |2.4        |2.0        |3.0     |3.0 |
|67.0|0.0 |3.0      |115.0          |564.0     |0.0             |2.0                  |160.0      |0.0   |1.6        |2.0        |0.0     |7.0 |
|57.0|1.0 |2.0      |124.0          |261.0     |0.0             |0.0                  |141.0      |0.0   |0.3        |1.0        |0.0     |7.0 |
|64.0|1.0 |4.0      |128.0          |263.0     |0.0             |0.0                  |105.0      |1.0   |0.2        |2.0        |

In [540]:
df.count(), len(df.columns)

(270, 13)

In [541]:
df.dtypes

[('año', 'double'),
 ('sexo', 'double'),
 ('tipoDolor', 'double'),
 ('presionDescanso', 'double'),
 ('colesterol', 'double'),
 ('azucarSangreL120', 'double'),
 ('electrocardioDescanso', 'double'),
 ('frecCardMax', 'double'),
 ('angina', 'double'),
 ('depresionST', 'double'),
 ('pendienteST', 'double'),
 ('numVasos', 'double'),
 ('thal', 'double')]

# Escriba esta función ENFERMO() para marcar 0 como negativo y 1 como positivo, porque la regresión logística binaria requiere uno de dos resultados. También debe crear el dataframe de Spark raw_data usando la operación transform() y seleccionando solo la columna de características.

In [542]:
# alternativa utilizando withColumn-when
df = df.withColumn(
    'sick',
    when((df['thal'] == 3) | (df['thal'] == 6), 0).otherwise(1)
)
df.show(5, truncate=False)

+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+----+
|año |sexo|tipoDolor|presionDescanso|colesterol|azucarSangreL120|electrocardioDescanso|frecCardMax|angina|depresionST|pendienteST|numVasos|thal|sick|
+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+----+
|70.0|1.0 |4.0      |130.0          |322.0     |0.0             |2.0                  |109.0      |0.0   |2.4        |2.0        |3.0     |3.0 |0   |
|67.0|0.0 |3.0      |115.0          |564.0     |0.0             |2.0                  |160.0      |0.0   |1.6        |2.0        |0.0     |7.0 |1   |
|57.0|1.0 |2.0      |124.0          |261.0     |0.0             |0.0                  |141.0      |0.0   |0.3        |1.0        |0.0     |7.0 |1   |
|64.0|1.0 |4.0      |128.0          |263.0     |0.0             |0.0                  |105.0      |1

In [543]:
# alternativa utilizando transform-lambda
df = df.transform(
    lambda df: df.withColumn(
        'enfermo',
        when((df['thal'] == 3) | (df['thal'] == 6), 0).otherwise(1))
)
df.show(5)

+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+----+-------+
| año|sexo|tipoDolor|presionDescanso|colesterol|azucarSangreL120|electrocardioDescanso|frecCardMax|angina|depresionST|pendienteST|numVasos|thal|sick|enfermo|
+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+----+-------+
|70.0| 1.0|      4.0|          130.0|     322.0|             0.0|                  2.0|      109.0|   0.0|        2.4|        2.0|     3.0| 3.0|   0|      0|
|67.0| 0.0|      3.0|          115.0|     564.0|             0.0|                  2.0|      160.0|   0.0|        1.6|        2.0|     0.0| 7.0|   1|      1|
|57.0| 1.0|      2.0|          124.0|     261.0|             0.0|                  0.0|      141.0|   0.0|        0.3|        1.0|     0.0| 7.0|   1|      1|
|64.0| 1.0|      4.0|          128.0|     263.0|    

In [544]:
# drop newSick column creada en alternativa 1
df = df.drop('sick')
df.show(5, truncate=False)

+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+-------+
|año |sexo|tipoDolor|presionDescanso|colesterol|azucarSangreL120|electrocardioDescanso|frecCardMax|angina|depresionST|pendienteST|numVasos|thal|enfermo|
+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+-------+
|70.0|1.0 |4.0      |130.0          |322.0     |0.0             |2.0                  |109.0      |0.0   |2.4        |2.0        |3.0     |3.0 |0      |
|67.0|0.0 |3.0      |115.0          |564.0     |0.0             |2.0                  |160.0      |0.0   |1.6        |2.0        |0.0     |7.0 |1      |
|57.0|1.0 |2.0      |124.0          |261.0     |0.0             |0.0                  |141.0      |0.0   |0.3        |1.0        |0.0     |7.0 |1      |
|64.0|1.0 |4.0      |128.0          |263.0     |0.0             |0.0              

In [545]:
# Assemble all the features into a single vector
# se excluye la variable "thal"
assembler = VectorAssembler(
    inputCols=[
        'año', 'sexo', 'tipoDolor', 'presionDescanso',
        'colesterol', 'azucarSangreL120', 'electrocardioDescanso',
        'frecCardMax', 'angina', 'depresionST', 'pendienteST',
        'numVasos'
      ],
    outputCol='caracteristicas'
)

df = assembler.transform(df)

In [546]:
df.show(5, truncate=False)

+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+-------+--------------------------------------------------------+
|año |sexo|tipoDolor|presionDescanso|colesterol|azucarSangreL120|electrocardioDescanso|frecCardMax|angina|depresionST|pendienteST|numVasos|thal|enfermo|caracteristicas                                         |
+----+----+---------+---------------+----------+----------------+---------------------+-----------+------+-----------+-----------+--------+----+-------+--------------------------------------------------------+
|70.0|1.0 |4.0      |130.0          |322.0     |0.0             |2.0                  |109.0      |0.0   |2.4        |2.0        |3.0     |3.0 |0      |[70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0]|
|67.0|0.0 |3.0      |115.0          |564.0     |0.0             |2.0                  |160.0      |0.0   |1.6        |2.0        |0.0     |7.0 |1      |[67.0,0.

# Selecciono sólo las columnas "caracteristicas" y "enfermo" para el modelo

In [547]:
# Select only the 'features' and 'new_column' for the model
model_data = df.select('caracteristicas', 'enfermo')
model_data.show(3, truncate=False)

+--------------------------------------------------------+-------+
|caracteristicas                                         |enfermo|
+--------------------------------------------------------+-------+
|[70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0]|0      |
|[67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0]|1      |
|[57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0]|1      |
+--------------------------------------------------------+-------+
only showing top 3 rows



# Use Standard Scaler para poner todos los números en la misma escala.
Esto toma la observación y resta la media, y luego la divide por la
desviación estándar.

In [548]:
scaler = StandardScaler(inputCol='caracteristicas', outputCol='features', withMean=True, withStd=True)
scalerModel = scaler.fit(model_data)
datos_escalados = scalerModel.transform(model_data)

# Imprima el estado actual de sus datos

In [549]:
datos_escalados.show(3, truncate=False)

+--------------------------------------------------------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|caracteristicas                                         |enfermo|features                                                                                                                                                                                                                                      |
+--------------------------------------------------------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0]|0      |[1.7089200771370

# Divida los datos en conjuntos de datos de entrenamiento y prueba. Use 50/50.

In [550]:
# Split the data into training and test sets
train_data, test_data = datos_escalados.randomSplit([0.5, 0.5])

In [551]:
# Create a Logistic Regression model and fit it to the training data
lr = LogisticRegression(featuresCol='features', labelCol='enfermo')
lr_model = lr.fit(train_data)

# Make predictions on the test data
predictions = lr_model.transform(test_data)

# Show some predictions
predictions.show()

+--------------------+-------+--------------------+--------------------+--------------------+----------+
|     caracteristicas|enfermo|            features|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+--------------------+----------+
|(12,[0,2,3,4,7,10...|      0|[-1.9138441120828...|[4.28207049863806...|[0.98637419673767...|       0.0|
|(12,[0,2,3,4,7,10...|      0|[-1.4747211800561...|[4.19354586602046...|[0.98513172867920...|       0.0|
|(12,[0,2,3,4,7,10...|      0|[-1.3649404470495...|[4.08714096944206...|[0.98348999606023...|       0.0|
|(12,[0,2,3,4,7,10...|      0|[-1.0355982480295...|[6.25305164528191...|[0.99807912480816...|       0.0|
|[35.0,0.0,4.0,138...|      0|[-2.1334055780961...|[2.81409478081402...|[0.94343274421177...|       0.0|
|[35.0,1.0,4.0,120...|      1|[-2.1334055780961...|[0.10126552521380...|[0.52529476912730...|       0.0|
|[35.0,1.0,4.0,126...|      1|[-2.1334055780961...|[0.1

# Finalmente Use la función F de Spark SQL para crear una nueva columna correcta cuando ENFERMO() es igual a la predicción, lo que significa que el resultado predicho es igual a los resultados reales.

In [552]:
resultado = predictions.withColumn('resultado_prediccion', when(predictions.enfermo == predictions.prediction , "correcta").otherwise("incorrecta"))
resultado.show()

+--------------------+-------+--------------------+--------------------+--------------------+----------+--------------------+
|     caracteristicas|enfermo|            features|       rawPrediction|         probability|prediction|resultado_prediccion|
+--------------------+-------+--------------------+--------------------+--------------------+----------+--------------------+
|(12,[0,2,3,4,7,10...|      0|[-1.9138441120828...|[4.28207049863806...|[0.98637419673767...|       0.0|            correcta|
|(12,[0,2,3,4,7,10...|      0|[-1.4747211800561...|[4.19354586602046...|[0.98513172867920...|       0.0|            correcta|
|(12,[0,2,3,4,7,10...|      0|[-1.3649404470495...|[4.08714096944206...|[0.98348999606023...|       0.0|            correcta|
|(12,[0,2,3,4,7,10...|      0|[-1.0355982480295...|[6.25305164528191...|[0.99807912480816...|       0.0|            correcta|
|[35.0,0.0,4.0,138...|      0|[-2.1334055780961...|[2.81409478081402...|[0.94343274421177...|       0.0|            co

In [553]:
# resumen resultado predicción
num_predicciones=resultado.count()
print(f"Total predicciones: {num_predicciones}")
lista=resultado.select('resultado_prediccion').collect()
r=dict(zip(lista,map(lambda x: lista.count(x),lista)))
for k,v in r.items():
  print(f"{k[0]} = {v} es un {v/num_predicciones*100:.2f}% de las predicciones")

Total predicciones: 144
correcta = 93 es un 64.58% de las predicciones
incorrecta = 51 es un 35.42% de las predicciones


In [554]:
spark.stop()